In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [3]:
!git pull https://github.com/rohit-khoiwal-30/msgmt.git

remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.
From https://github.com/rohit-khoiwal-30/msgmt
 * branch            HEAD       -> FETCH_HEAD
Updating a5224bf..293f5ec
Fast-forward
 useEUCL-checkpoint.ipynb | 234 +++++++++----------
 useTripleLoss .ipynb     | 567 ++++++++++++++++++++++++++++++++++-------------
 2 files changed, 524 insertions(+), 277 deletions(-)


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("evaluation.csv")

In [6]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input
from keras.regularizers import l2

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install -q keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

In [7]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [8]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, nonreason_batch, y_hat = [], [], [], []
    all_groups = list(range(in_groups[0].shape[0]))

    group_idx = np.random.choice(all_groups, size = batch_halfsize*2)
    text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
    
    b_group_idx = group_idx
    y_hat += [1]*batch_halfsize
    reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
    

    non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
    b_group_idx = non_group_idx
    y_hat += [0]*batch_halfsize
    nonreason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(nonreason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, nonreason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack, nonreason_stack], y_hatstack


def identity_loss(y_true, y_pred):
    return K.mean(y_pred)

def triplet_loss(x, alpha = 0.2):
    # Triplet Loss function.
    anchor,positive,negative = x
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)
    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
    return loss

In [9]:
def accuracy(y_true, y_pred):
    results = (y_pred >= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [10]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [47]:
def encoder1(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dense(128, activation="linear", kernel_regularizer=l2(1e-3))(embedding_output)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(a)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    outputs = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    # outputs = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)
    # a = tfl.BatchNormalization()(a)
    # outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

def encoder2(name):
    inputs = Input(shape=(), dtype=tf.string)

    shared_embedding_layer = hub.KerasLayer(fine_tuned_module_object, trainable=True, name=name)
    embedding_output= shared_embedding_layer(inputs)
    a = tfl.Dense(64, activation="linear", kernel_regularizer=l2(1e-3))(embedding_output)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    outputs = tfl.Dense(32, activation="linear", kernel_regularizer=l2(1e-3))(a)
    # outputs = tfl.Lambda(lambda x: K.l2_normalize(x,axis=-1))(a)
    # a = tfl.BatchNormalization()(a)
    # outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

In [48]:
def get_model():
    #textEncoder
    inputText = Input(shape=(), dtype=tf.string)
    textModel = encoder1("textBertEncoder")
    text_embedd = textModel(inputText)

    #reasonEncoder
    reasonModel = encoder2("reasonBertEncoder")

    inputReason = Input(shape=(), dtype=tf.string)
    reason_embedd = reasonModel(inputReason)

    #nonReasonEncoder
    inputNotReason = Input(shape=(), dtype=tf.string)
    nonreason_embedd = reasonModel(inputNotReason)

    loss = tfl.Lambda(triplet_loss)([text_embedd, reason_embedd, nonreason_embedd])

    model = Model(inputs = [inputText, inputReason, inputNotReason], outputs = [loss], name="bertModel")
    return model, textModel, reasonModel

In [49]:
model, textModel, reasonModel = get_model()

In [50]:
model.summary()

Model: "bertModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_24 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_27 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_28 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 model_6 (Functional)           (None, 32)           256874592   ['input_24[0][0]']               
                                                                                          

In [51]:
model.compile(optimizer="adam", loss = identity_loss)

In [52]:
eval_df = df_test[df_test['label'] == 1]

In [53]:
loss_history = model.fit(siam_gen([df["text"], df["reason"]], 64), 
                         validation_data = siam_gen([eval_df["text"].values, eval_df["reason"].values], 64),
                         steps_per_epoch=50, validation_steps=10, epochs = 30, 
                         verbose = True, use_multiprocessing=True)

Epoch 1/30
50/50 [==============================] - 18s 224ms/step - loss: 0.8133 - val_loss: 0.6659
Epoch 2/30
50/50 [==============================] - 10s 194ms/step - loss: 0.5302 - val_loss: 0.6287
Epoch 3/30
50/50 [==============================] - 10s 195ms/step - loss: 0.5044 - val_loss: 0.5885
Epoch 4/30
50/50 [==============================] - 11s 220ms/step - loss: 0.4747 - val_loss: 0.6375
Epoch 5/30
50/50 [==============================] - 10s 202ms/step - loss: 0.4581 - val_loss: 0.8255
Epoch 6/30
50/50 [==============================] - 10s 194ms/step - loss: 0.4291 - val_loss: 1.0405
Epoch 7/30
50/50 [==============================] - 10s 195ms/step - loss: 0.4154 - val_loss: 1.3647
Epoch 8/30
50/50 [==============================] - 10s 196ms/step - loss: 0.4029 - val_loss: 1.1746
Epoch 9/30
50/50 [==============================] - 10s 195ms/step - loss: 0.3902 - val_loss: 1.1221
Epoch 10/30
50/50 [==============================] - 10s 204ms/step - loss: 0.3784 - val_lo

In [54]:
left_input = tf.keras.Input(shape=(), dtype=tf.string)
right_input = tf.keras.Input(shape=(), dtype=tf.string)

textEmbedding = textModel(left_input)
reasonEmbedding = reasonModel(right_input)

cosine_similiarity=tf.keras.layers.Dot(axes=-1,normalize=True)([textEmbedding, reasonEmbedding])


classifier_model = Model(inputs = [left_input, right_input], outputs = [cosine_similiarity], name="classifier_model")

In [55]:
classifier_model.summary()

Model: "classifier_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_29 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_30 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 model_6 (Functional)           (None, 32)           256874592   ['input_29[0][0]']               
                                                                                                  
 model_7 (Functional)           (None, 32)           256832992   ['input_30[0][0]']               
                                                                                   

In [56]:
y_preds = classifier_model.predict([df['text'], df['reason']])
accuracy(df["label"], y_preds)

0.9044153323629306

In [57]:
y_preds = classifier_model.predict([df_test['text'], df_test['reason']])
accuracy(df_test["label"], y_preds)

0.5528888888888889